<a href="https://colab.research.google.com/github/Junghyun1217/ImpactFlow-Streamlined-automation-for-social-value-evaluation/blob/main/ImpactFlow(langchain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pararrel
# 표준 라이브러리
import getpass
import os
import json
import re
import glob
import time
from pathlib import Path
from typing import List, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed

# 데이터 시각화 라이브러리
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patches as mpatches
import matplotlib.patheffects as patheffects
from matplotlib.patches import Rectangle
import seaborn as sns

# LangChain 및 AI 관련 라이브러리
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from langchain.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# import gethwp

load_dotenv(".env", override=True)

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LLAMA_CLOUD_API_KEY")

# print(os.environ.get("OPENAI_API_KEY")[:10])
# print(os.environ.get("LLAMA_CLOUD_API_KEY")[:10])


DOCUMENT_PATH = "docs"
## check
SDGS_SUBGOALS_PATH = 'data/SDGs_json/SDGs_subgoals.json'
GRI_DETAIL_PATH = 'data/gri/gri_detail.json' # <-- 이 줄을 추가하세요

MODEL_NAME = "gpt-4o-mini"  # 유효한 모델명으로 수정
TOPN_SELECT = 3
MAX_WORKERS = 8 # 병렬 처리 스레드 개수 (필요에 맞게 조절)

## check
# with open('reference/SDGs_subgoals.json', 'r') as f:
with open('data/SDGs_json/SDGs_subgoals.json', 'r', encoding='utf-8') as f:
    SDGS_SUBGOALS = json.load(f)

# llm = ChatOpenAI(model="gpt-4.1", temperature = 0.1)
llm = ChatOpenAI(model="gpt-5-mini")




# =================================================================
# 섹션 A: SDGs 분석 관련 함수들
# =================================================================
#=====================================================#
#================ Step1: 기업 정보 입력 ==================#
#=====================================================#
def process_file(file_path, ext, parser):
    try:
        if ext == "hwp":
            docs = gethwp.read_hwp(file_path)
        elif ext == "hwpx":
            docs = gethwp.read_hwpx(file_path)
        else:
            docs = parser.load_data(file_path=file_path)
        texts = ""
        for doc in docs:
            texts += doc.text
        return texts
    except Exception as e:
        print(f"[WARN] {file_path} 읽기 실패: {e}")
        return ""

def read_document(file_dir):
    print('='*100)
    print("Step1: 기업 정보 입력")

    parser = LlamaParse(
        use_vendor_multimodal_model=True,
        vendor_multimodal_model_name="openai-gpt-4-1-mini",
        vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
        result_type="text",
        language="ko",
    )

    target_exts = ["pdf", "pptx", "doc", "docx", "hwp", "hwpx"]
    file_tasks = []

    for ext in target_exts:
        file_paths = glob.glob(os.path.join(file_dir, f"*.{ext}"))
        for file_path in file_paths:
            file_tasks.append((file_path, ext))

    company_info = ""
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_file, fp, ext, parser) for fp, ext in file_tasks]
        for future in futures:
            company_info += future.result()

    return company_info
#=====================================================#
#================ Step2: 기업 정보 마스킹 ================#
#=====================================================#

def mask_info(company_info):
    print('='*100)
    print("Step2: 기업 정보 마스킹 (현재 그대로 return)")
    return company_info

#=====================================================#
#================ Step3: 기업 정보 요약 ================#
#=====================================================#

def summary_sv(text: str) -> str:
    print('='*100)
    print("Step3: 기업 정보 요약")
    """
    주어진 보고서 문단을 분석하여 기업 또는 단체가 창출하는 사회적 가치를 한 문단으로 간결하게 요약합니다.
    요약 시 다음 요소를 포함합니다:
      - 해결하려는 사회문제
      - 제공하는 솔루션
      - 기대되는 사회적 효과
    불필요한 수식어나 장황한 설명은 제외하고, 핵심만 명확하게 기술합니다.
    """
    prompt = f"""
    아래의 문단을 읽고, 기업 또는 단체가 창출하는 사회적 가치를 한 문장 또는 두 문장으로 간결하게 요약하시오.
    반드시 다음 요소를 포함하십시오:
    1) 해결하려는 사회문제
    2) 제공하는 솔루션 또는 활동
    3) 기대되는 사회적·환경적 효과
    주관적 평가나 불필요한 수식어는 배제하고, 사실 기반으로 작성할 것.

예시: "그린인프라스트럭처는 도심 내 유휴부지에 도시농업과 작은 숲 조성 프로젝트를 시행해 미세먼지 저감과 주민 휴식공간 확대를 추진합니다. 지역 주민과 함께 관리계획을 수립해 생물다양성 회복과 공공 공간의 사회적 가치를 높입니다."

[문단]
{text}
    """
    response = llm.invoke(prompt,seed=23).content
    return response



#=====================================================#
#============= Step4:SDGs Main Code 판별 ==============#
#=====================================================#
def match_sdgs_by_llm(text: str) -> dict:
    print('='*100)
    print("Step4:SDGs Main Code 판별")

    """
    GPT가 보고서 문단을 기반으로 가장 유사한 SDGs 목표를 1~2개 추론합니다.
    각 코드에 대해 간단한 이유와 함께 설명을 JSON 형식으로 반환합니다.
    예: {"2": "기아 종식 — 아동 영양 문제 해소와 관련 있음"}
    """

    sdgs_definitions = """
    SDGs 정의 목록:
    - SDGs 1: 모든 곳에서 모든 형태의 빈곤 종식
    - SDGs 2: 기아 종식, 안전하고 영양이 개선된 식량 달성, 지속가능한 농업 발전
    - SDGs 3: 모든 연령층을 위한 건강한 삶 보장과 웰빙 증진
    - SDGs 4: 포용적이고 공평한 양질의 교육을 보장하고 모두를 위한 평생 학습 기회를 촉진합니다
    - SDGs 5: 성평등 달성과 모든 여성 및 여아의 권익신장
    - SDGs 6: 모두를 위한 물과 위생의 이용가능성과 지속가능한 관리 보장
    - SDGs 7: 모두를 위한 적정가격의 신뢰할 수 있고 지속가능하며 현대적인 에너지에 대한 접근 보장
    - SDGs 8: 지속적이고 포용적이며 지속 가능한 경제 성장, 완전하고 지속 가능한 생산적인 고용과 모두를 위한 양질의 일자리 증진
    - SDGs 9: 회복력 있는 사회기반시설 구축, 포용적이고 지속가능한 산업화 증진과 혁신 도모
    - SDGs 10: 국가 내 및 국가 간 불평등 감소
    - SDGs 11: 포용적이고 안전하며 회복력 있고 지속가능한 도시와 주거지 조성
    - SDGs 12: 지속가능한 소비와 생산 양식의 보장
    - SDGs 13: 기후변화와 그로 인한 영향에 맞서기 위한 긴급 대응
    - SDGs 14: 지속가능발전을 위하여 대양, 바다, 해양자원의 보전과 지속가능한 이용
    - SDGs 15: 육상생태계 보호, 복원 및 지속가능한 이용 증진, 지속가능한 산림 관리, 사막화 방지, 토지황폐화 중지와 회복, 생물다양성 손실 중단
    - SDGs 16: 지속가능발전을 위한 평화롭고 포용적인 사회 증진, 모두에게 정의 보장과 모든 수준에서 효과적이고 책임성 있으며 포용적인 제도 구축
    - SDGs 17: 나머지 목표들의 실행수단 강화와 지속가능발전을 위한 글로벌 파트너십 재활성화
    """
    prompt = f"""

너는 사회적 가치 평가 및 SDGs 전문가야.
다음은 SDGs 목표 정의 목록이야: {sdgs_definitions}

아래 문단을 읽고, 이와 가장 관련 있는 SDGs 목표를 1~2개 추론해줘.
각 SDGs 코드(숫자)와 간단한 이유를 JSON 형식으로 반환해.

형식 예시:
{{
  "2": "기아 종식 — 아동 영양 문제 해소와 관련 있음",
  "3": "건강과 웰빙 — 신체 및 정신 건강 개선과 관련"
}}

[문단]
{text}
    """

    response = llm.invoke(prompt,seed=23).content
    return response

#=====================================================#
#============ Step5-1 :SDGs Sub Code 판별 =============#
#=====================================================#
def match_sdgs_sub_by_llm(selected_sdgs_goal: str) -> dict:
    print('='*100)
    print("Step5-1:SDGs Sub Code 판별")
    """
    이미 선택된 SDGs 대제목(예: {"2": "...", "3": "..."})에 대해,
    관련된 세부 목표(subgoal)들을 추론하고 JSON으로 반환합니다.
    """
    selected_sdgs_goal_dict = json.loads(selected_sdgs_goal)

    prompt = f"""
너는 SDGs 세부 목표 분석가야.

사용자는 다음과 같은 SDGs 대목표를 선택했어:
{selected_sdgs_goal_dict.keys()}

아래는 각 SDGs 대목표에 대한 세부 목표들이야:
{SDGS_SUBGOALS}

각 SDGs 대목표에 대해 가장 관련 있는 **세부 목표 코드(예: 2.1, 3.2 등)**와 그 이유를 1~2개씩 JSON 형식으로 알려줘.

형식 예시:
{{
  "2.1": "영양 공급 개선 — 아동 급식과 관련",
  "3.4": "정신 건강 증진 — 우울증 예방과 관련"
}}
    """

    response = llm.invoke(prompt,seed=23).content
    try:
        # GPT가 ```json ... ``` 로 감쌌을 가능성 대비
        cleaned = re.sub(r"```json|```", "", response).strip()
        return json.loads(cleaned)
    except Exception as e:
        print("⚠️ JSON 파싱 실패. 원본 응답:", response)
        return {"error": "JSON parsing failed", "raw_response": response}


#=====================================================#
#=============== Step5-2 : RAW 보고서 작성==============#
#=====================================================#


def generate_raw_report(company_info: str, sdgs_code: dict) -> str:
    print('='*100)
    print("Step5-2 :사회적 가치 창출 포인트 작성")
    """
    소셜벤처 기업 보고서를 작성하는 도우미로,
    제공된 기업 정보를 바탕으로 '사회적 가치 창출 포인트' 섹션을 생성합니다.
    """

    prompt = f"""
    너는 사회적 가치 분석 전문 컨설턴트야.
    이제 다음 기업의 사회 가치 창출 보고서에 들어갈 사회적 가치 보고서 내용을 다음 형식에 맞춰 작성해줘.

    📄 기업 정보:
    {company_info}

    📌 작성 지침:
    - 전체 5문단 구성("사회적 가치 창출 포인트", "WHAT", "WHO", "HOW MUCH", "RISK"로 총 5개의 문단)

    1. **사회적 가치 창출 포인트**
    - 기업이 해결하고자 하는 사회문제와, 제공하는 솔루션의 핵심을 분석해.
    - 솔루션이 확장될 경우 기대되는 사회적 효과까지 포함해서 작성해.
    - 예: "[기업]의 가치는 [문제]를 해결하기 위해 [솔루션]을 제공함으로써 창출됩니다. 이는 [사회적 효과]효과의 증대로 이어질 것으로 기대할 수 있습니다."


    2. **WHAT**,
    - 이 기업이 창출하는 긍정적인 변화(사회적 가치)의 유형을 자세히 설명하고, SDG {json.dumps(sdgs_code, ensure_ascii=False)}번과 연결지어 마무리해.
    - 환경, 교육, 건강, 고용 등 어떤 영역에서 어떤 가치가 발생하는지 구체적으로 설명해.
    - 예: ”[기업]는 [문제]를 해결하기 위해 [기술/서비스]를 제공하며, 이는 [대상]의 [사회적 효과]에 기여합니다. 이는 SDGs의 [관련 목표] 달성과도 연결됩니다.”


    3. **WHO**
    - 해당 사회적 가치가 **누구에게 영향을 미치는지** 설명해.
    - 대상 집단의 특성(예: 소외계층, 저소득층, 청년, 지역사회 등)과 어떤 방식으로 혜택을 받는지 명확히 작성해.
    - 예: ”[기업]는 [대상]이 겪는 [문제]를 해결하기 위해 [솔루션/서비스]을 제공합니다. 예를 들어, [문제의 구체적 상황]에 대응하여 [실행 방식]을 통해 [기대 효과 또는 변화]를 이끌어냅니다. 이는 [대상]의 삶의 질 향상과 함께 [사회적 가치] 창출로 이어지며, 지속가능한 사회 구현에 기여합니다.”


    4. **HOW MUCH**
    - 해당 기업의 임팩트(사회적 가치)를 정량적으로 측정 또는 산출할 수 있는 방식을 제시하되, 없는 숫자를 지어내서는 안돼.
    - 지표가 명확하지 않다면 규모·범위·지속성에 대한 추정도 가능해.
    - 예: ”[기업]의 임팩트는 [솔루션/서비스] 도입 전후의 변화를 정량적으로 측정함으로써 확인할 수 있습니다. 구체적으로는 [성과 측정 대상]에 대해 [비교 지표]를 산출하고, 이를 바탕으로 [사회적 성과]를 도출합니다."

    5. **RISK**
    - 사회적 가치가 실제로 실현되지 못할 **가능성 또는 제한사항**을 설명해.
    - 외부 요인, 정책 변화, 이해관계자 이슈 등 리스크 요인과 그 영향에 대해 분석해.
    - 필요시 리스크를 관리하기 위한 전략이나 방어 요소도 포함해.
    - 예: “[기업]의 사회적 가치 실현에는 [외부 요인]이 영향을 미칠 수 있습니다. 예를 들어, [구체적 위험 상황]이 발생하면 [성과 축소 또는 사업 차질]이 우려됩니다. 이를 완화하기 위해 [리스크 대응 전략]을 병행하고 있습니다.”


    🧾 출력 형식:
    실제 보고서 문단 형태로 바로 사용 가능하도록 작성해.
    각 문단은 제목 있게 순차적으로 이어지도록 작성해.
    """

    result = llm.invoke(prompt,seed=23).content
    return result

#=====================================================#
#============ Step6 :사실검증 및 조건부 해설 추가 ===========#
#=====================================================#

@tool
def verify_and_correct_facts(paragraph: str) -> str:
    """
    당신은 ESG 보고서의 사실 검증 전문가입니다.
    다음 문단이 ESG(환경, 사회, 지배구조) 관련 내용인지 우선 판단하세요.
    ESG 관련 내용일 경우에만 사실 여부를 엄밀히 검증합니다.
    ESG와 직접 관련 없는 내용이라면 수정하거나 해설하지 말고, 원문을 그대로 반환하세요.
    """
    print('검증 tool 작동.')
    prompt = f"""
    너는 ESG 보고서의 사실 검증 전문가입니다.
    다음 문단의 내용을 검증하세요.

    규칙:
    1. 보고서에 언급된 수치, 기간, 장소, 기관명, 사건 등 ESG 관련 사실 요소를 중점적으로 검토합니다.
    2. 사실이 정확하면 원문을 그대로 반환합니다.
    3. 부정확한 ESG 사실이 있으면, 가능한 범위 내에서 올바른 내용으로 수정하여 반환합니다.
    4. 검증 시 인터넷 지식과 일반 상식을 활용하되, 불확실한 정보는 추정하지 말고 원문 그대로 둡니다.
    5. 최종 출력은 문단 한 개로 하며, 추가 설명이나 해설은 포함하지 마세요.

    문단: "{paragraph}"
    """
    result = llm.invoke(prompt,seed=23).content.strip()
    return result

@tool
def interpret_esg_impact(statement: str) -> str:
    """
    ESG 성과를 직관적인 비유로 설명할 수 있는 경우에는 비유를 붙이고,
    비유가 적합하지 않으면 원문을 그대로 반환합니다.
    """
    print('비유 tool 작동.')

    sdgs_to_esg = {
        "1": "S", "2": "S", "3": "S", "4": "S", "5": "S",
        "6": "S", "7": "S", "8": "S", "9": "S", "10": "S",
        "11": "S", "12": "E", "13": "E", "14": "E", "15": "E",
        "16": "G", "17": "G"
    }

    guide_dict = {
        "E": "환경 보호 또는 자원 절감과 관련됨. 탄소저감, 플라스틱 절감, 에너지 효율 등에 대한 수치적·비유적 설명 가능 여부를 판단.",
        "S": "사회적 가치, 고용, 교육, 건강, 포용성과 관련됨. 사람들의 삶에 실질적 영향을 줄 수 있는 비유 가능 여부를 판단.",
        "G": "기업 지배구조, 윤리성, 투명성과 관련됨. 신뢰·투명성에 기여하는 효과를 비유 가능 여부를 판단.",
        "":  "ESG 카테고리 불분명. 그래도 직관적 설명 가능 여부를 판단."
    }

    # 1차 프롬프트: 비유 가능 여부 판단
    check_prompt = f"""
    다음 성과 문장이 일반 대중이 직관적으로 이해할 수 있는 **실생활 비유**로 표현 가능한지 판단하세요.
    - 가능하다면 "YES"만 출력
    - 어렵거나 부적합하면 "NO"만 출력
    문장: "{statement}"
    """
    can_metaphor = llm.invoke(check_prompt).content.strip().upper()

    if can_metaphor != "YES":
        # 비유 불가 → 원문 그대로 반환
        return statement

    # 2차 프롬프트: 비유 생성
    prompt = f"""
    너는 ESG 성과를 일반 대중이 명확하고 이해하기 쉽게 설명하는 전문가입니다.

    조건:
    - 복잡한 표현은 피하고,
    - 일반 대중이 직관적으로 이해할 수 있도록 **실생활 비유** 또는 **수치적 환산** 포함
    - 보고서에 명시된 수치만 사용 (없는 경우 정량 표현 생략)
    - 문장은 1~2줄
    - 과도한 감성 표현 금지


    문장: "{statement}"

    출력 예시:
    - "이는 30년생 소나무 약 100만 그루를 심은 효과와 같습니다."
    - "이는 한 마을의 모든 가정에 안정적인 일자리를 제공한 셈입니다."
    """
    result = llm.invoke(prompt,seed=23).content
    statement_with_explanation = statement + '\n\nAI해설: ' + result
    return statement_with_explanation

agent = create_react_agent(
    model=llm,
    tools=[interpret_esg_impact, verify_and_correct_facts],
        prompt="""
        너는 ESG 보고서 분석 전문가 에이전트야.
        두 가지 역할을 수행해야 해:
        1) ESG 문단의 사실 여부를 검증하고, 필요시 올바르게 수정해줘.
        2) 사실이 검증된 문단에서, 비유가 가능하다면, 대중이 쉽게 이해할 수 있도록 실생활 비유나 직관적 설명을 1~2줄로 덧붙여줘. 단, 비유가 적절하지 않으면 원문을 그대로 반환해야 해.
        항상 정확하고 간결하게 응답해.
        툴을 효과적으로 사용해서 문단별로 처리해줘.
        """
    )


#=====================================================#
#Step6 :사회적 가치 창출 포인트에 대한 사실검증 및 조건부 해설 추가#
#=====================================================#

def process_raw_report_agent(report_text: str) -> dict:
    print('='*100)
    print("Step6 :사회적 가치 창출 포인트에 대한 사실검증 및 조건부 해설 추가")
    paragraphs = [p.strip() for p in report_text.split("\n\n") if p.strip()]

    paragraph = paragraphs[0]
    # 1단계: 사실검증 및 수정
    verify_instruction = (
        "당신은 ESG 보고서 문단의 사실을 검증하는 역할을 맡았습니다.\n"
        "아래 문단이 사실에 부합하는지 확인하고, 부정확하면 정확하게 수정해 주세요.\n"
        "수정하지 않아도 되면 그대로 반환해 주세요.\n"
        "다른 설명, 레이블, 수식어, 이유는 절대 붙이지 말고, 수정된 문단 내용만 그대로 출력하세요.\n"
        "잘못된 예시: 수정된 문장: 이 기업은...\n"
        "올바른 예시: 이 기업은...\n\n"
        "문단:\n" + paragraph
    )
    verify_response = agent.invoke(
        {"messages": [HumanMessage(content=verify_instruction)]}
    )

    messages = verify_response.get('messages', [])
    if messages:
        last_message = messages[-1]
        corrected_paragraph = last_message.content.strip()
    else:
        corrected_paragraph = paragraph


    # 2단계: 비유 생성
    interpret_instruction = (
        "당신은 ESG 성과를 대중이 쉽게 이해하도록 비유하는 문장을 생성할 수 있을지 판단해야 합니다.\n"
        "비유가 가능하면 **비유 문장만** 반환하세요.\n"
        "비유가 불가능하면 **None을** 반환하세요.\n"
        "비유 문장은 불필요한 서두나 이유 설명 없이 한 문장으로 작성하세요.\n"
        "원문 문단은 절대 수정하거나 재출력하지 마세요.\n\n"

        "예시 1:\n"
        "문장: 이 기업은 지난 1년간 에너지 사용량을 25% 감축하여 환경 보호에 기여했습니다.\n"
        "비유: 이는 5,000가구가 한 해 동안 사용하는 전기를 절감한 것과 같습니다.\n\n"

        "예시 2:\n"
        "문장: 이 회사는 직원 다양성과 포용성을 증진하기 위한 정책을 강화했습니다.\n"
        "비유: None\n\n"  # 비유 불가 시 None 반환 예시

        "예시 3:\n"
        "문장: 본 기업의 사회공헌 프로그램은 지역사회 교육 지원에 집중하여 수백 명의 학생들에게 양질의 교육 기회를 제공했습니다.\n"
        "비유: 이는 한 도시의 모든 초등학교 학생들에게 맞춤형 교육을 제공한 효과와 같습니다.\n\n"

        f"문단:\n{corrected_paragraph}"
    )


    interpret_response = agent.invoke(
        {"messages": [HumanMessage(content=interpret_instruction)]}
    )

    messages = interpret_response.get('messages', [])
    if messages:
        last_message = messages[-1]
        ai_explain = last_message.content.strip()
    else:
        ai_explain = corrected_paragraph.strip()

    print('ai_explain:')
    print(ai_explain)

    paragraphs[0] = corrected_paragraph
    return ai_explain, paragraphs

def summarize_validated_report(validated_report: str, selected_sdgs_code: str) -> str:
    sdgs_main_code = json.loads(selected_sdgs_code)
    sdgs_main_code_str = ','.join(map(str, sdgs_main_code.keys()))

    prompt = (
        "제공된 ESG 보고서 내용을 자연스럽고 자세히 설명하는 문단을 만들어 주세요.\n"
        f"기업 소개, SDG 번호 {sdgs_main_code_str}, 그리고 사회적 가치에 대해 연속적으로 서술해 주세요.\n"
        "다음 양식에 맞춰서, 불필요한 내용은 제외하고 핵심만 명확하게 전달하세요.\n\n"
        "양식:\n"
        "[기업]은 [ESG 산업 분야] 산업에서 선도적인 역할을 하는 기업으로, SDG {sdgs_main_code_str}번에 부합합니다. "
        "[기업]은 [주요 활동/전략]를 통해 [사회적/환경적 가치]에 기여하며, [추가 활동]를 통해 지속가능한 미래를 추구하고 있습니다.\n\n"
        f"{validated_report}\n"
    )

    response = llm.invoke(prompt, seed=23)
    return response.content.strip()

def to_json(company_summary, sdgs_main, sdgs_sub, gri_main, ai_explain, validated_report_list, ai_summary):
    final_result = {
        'company_summary': company_summary,
        'sdgs_main': json.loads(sdgs_main),
        'sdgs_sub': sdgs_sub,
        'gri_main': gri_main  # <-- 여기에 GRI 분석 결과가 추가됩니다.
    }

    paragraphs = validated_report_list

    def safe_extract(index, keyword):
        try:
            return paragraphs[index].split(keyword, 1)[-1].strip()
        except Exception:
            return paragraphs[index] if index < len(paragraphs) else ""

    final_result['social_value_point'] = safe_extract(0, '사회적 가치 창출 포인트')
    final_result['ai_explain'] = ai_explain if ai_explain and ai_explain.strip().upper() != 'NONE' else 'None'
    final_result['what'] = safe_extract(1, 'WHAT')
    final_result['who'] = safe_extract(2, 'WHO')
    final_result['how_much'] = safe_extract(3, 'HOW MUCH')
    final_result['risk'] = safe_extract(4, 'RISK')
    final_result['ai_summary'] = ai_summary

    return final_result

# =================================================================
# 섹션 B: GRI 분석 관련 함수들
# =================================================================

# GRI 상위 항목에 대한 명확한 '정의'를 추가하여 LLM의 혼동을 방지합니다.
GRI_TOP_DEFINITIONS = [
    {"code": "201", "name": "경제 성과", "description": "조직의 경제적 성과와 그것이 이해관계자에게 미치는 영향에 대한 표준."},
    {"code": "202", "name": "시장 지위", "description": "경쟁, 독점금지, 공정 경쟁과 관련된 조직의 행동에 대한 표준."},
    {"code": "203", "name": "간접 경제 영향", "description": "지역 사회 개발 및 인프라 투자와 같은 조직의 간접적인 경제적 영향에 대한 표준."},
    {"code": "204", "name": "조달 관행", "description": "공급업체 선정 및 현지 조달과 같은 조직의 조달 관행에 대한 표준."},
    {"code": "205", "name": "반부패", "description": "뇌물 및 부패 방지를 위한 조직의 정책 및 관행에 대한 표준."},
    {"code": "206", "name": "반경쟁적 행위", "description": "가격 담합, 카르텔 등 반경쟁적 행위에 대한 조직의 법적 조치에 대한 표준."},
    {"code": "301", "name": "원자재", "description": "제품 및 서비스 생산에 사용되는 원자재의 무게 또는 부피에 대한 표준."},
    {"code": "302", "name": "에너지", "description": "조직 내 에너지 소비량 및 에너지 효율 개선 노력에 대한 표준."},
    {"code": "303", "name": "수자원 및 폐수", "description": "조직의 물 사용량, 폐수 배출 및 수자원 관리에 대한 표준."},
    {"code": "304", "name": "생물다양성", "description": "보호 지역 안팎에서 생물다양성에 미치는 조직의 영향에 대한 표준."},
    {"code": "305", "name": "온실가스 배출", "description": "조직의 온실가스(GHG) 직접 및 간접 배출량에 대한 표준."},
    {"code": "306", "name": "폐기물", "description": "조직의 폐기물 발생량 및 처리 방법에 대한 표준."},
    {"code": "308", "name": "공급업체 환경 평가", "description": "환경 기준을 사용하여 신규 및 기존 공급업체를 평가하는 조직의 관행에 대한 표준."},
    {"code": "401", "name": "고용", "description": "조직의 직원 채용, 이직, 복리후생 등 직접적인 고용 관행에 대한 표준. (주요 대상: 직원)"},
    {"code": "402", "name": "노사 관계", "description": "단체 교섭 협약에 포함된 직원 비율 등 조직의 노사 관계에 대한 표준. (주요 대상: 직원)"},
    {"code": "403", "name": "산업안전보건", "description": "조직의 작업장 내 안전보건 시스템 및 재해율에 대한 표준. (주요 대상: 직원)"},
    {"code": "404", "name": "훈련 및 교육", "description": "직원의 기술 개발 및 경력 개발을 위한 교육 프로그램에 대한 표준. (주요 대상: 직원)"},
    {"code": "405", "name": "다양성 및 동등한 기회", "description": "조직 내 성별, 연령 등 다양성 및 차별 금지에 대한 표준. (주요 대상: 직원)"},
    {"code": "406", "name": "차별 금지", "description": "차별 사건 및 시정 조치에 대한 표준. (주요 대상: 직원 및 이해관계자)"},
    {"code": "407", "name": "결사의 자유 및 단체교섭권", "description": "결사의 자유와 단체교섭권이 침해될 위험이 있는 사업장에 대한 표준."},
    {"code": "408", "name": "아동 노동", "description": "아동 노동의 위험이 있는 사업장 및 아동 노동 방지 정책에 대한 표준."},
    {"code": "409", "name": "강제 또는 의무 노동", "description": "강제 또는 의무 노동의 위험이 있는 사업장 및 관련 방지 정책에 대한 표준."},
    {"code": "412", "name": "인권 평가", "description": "인권 조항을 포함하는 계약 및 인권 평가를 받은 사업장에 대한 표준."},
    {"code": "413", "name": "지역 사회", "description": "지역 사회 참여, 영향 평가 및 개발 프로그램에 대한 표준. (주요 대상: 지역사회)"},
    {"code": "414", "name": "공급업체 사회 평가", "description": "사회적 기준을 사용하여 신규 및 기존 공급업체를 평가하는 조직의 관행에 대한 표준."},
    {"code": "415", "name": "공공 정책", "description": "로비 및 정치 기부금 등 공공 정책에 대한 조직의 참여에 대한 표준."},
    {"code": "416", "name": "고객 건강 및 안전", "description": "제품 및 서비스가 고객의 건강과 안전에 미치는 영향 및 관련 평가에 대한 표준. (주요 대상: 고객)"},
    {"code": "417", "name": "마케팅 및 라벨링", "description": "제품 정보, 라벨링, 마케팅 커뮤니케이션의 정확성 및 투명성에 대한 표준. (주요 대상: 고객)"},
    {"code": "418", "name": "고객 개인정보보호", "description": "고객 데이터의 개인정보보호 침해 및 유출 관련 불만 사항에 대한 표준. (주요 대상: 고객)"}
]

# =========================
# 2) 공통 유틸 및 데이터 구조 (이전과 동일)
# =========================
def load_json(path: str):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다. 경로: {path}")
        return None
    except json.JSONDecodeError:
        print(f"오류: JSON 형식이 올바르지 않습니다. 파일: {path}")
        return None

def normalize(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"[\u200b\ufeff]", "", text)
    return re.sub(r"\s+", " ", text).strip()

def find_sub_items(top_item_code: str, detail_data: dict) -> list[Document]:
    sub_items = []
    for key, value in detail_data.items():
        if key.startswith(top_item_code):
            doc = Document(page_content=f"{key}: {value}", metadata={"gri_code": key})
            sub_items.append(doc)
    return sub_items

class TopItem(BaseModel):
    item_code: str = Field(description="선택된 상위 항목의 코드 (예: '401')")
    item_name: str = Field(description="선택된 상위 항목의 이름 (예: '고용')")
    reason: str = Field(description="이 항목을 선택한 간략한 이유")

class TopItemList(BaseModel):
    top_items: list[TopItem] = Field(description="가장 관련성 높은 상위 항목 리스트")

# =========================
# 3) 단계별 분석 함수
# =========================
# ★★★★★ 여기가 핵심 수정 사항 (2) ★★★★★
def run_stage1_find_top_items(biz_context: str, gri_definitions: list, model_name: str, top_n: int) -> dict:
    """1단계: LLM을 통해 상위 GRI 항목을 판별합니다."""
    print('='*100)
    print("Step5-3: 상위 GRI 항목 판별")
    print("--- 1단계: 관련성 높은 상위 GRI 항목 판별 시작 ---")

    top_level_prompt_template = """
    당신은 최고 수준의 ESG 컨설턴트입니다. 주어진 사업 내용과 GRI 표준 정의 목록을 분석하여, 가장 직접적으로 관련된 GRI 상위 항목을 {top_n}개 선택해야 합니다.

    다음 3단계 사고 과정에 따라 분석하고 답변해주세요.

    1.  **핵심 영향 분석**: 이 사업의 핵심 활동은 무엇이며, 그로 인해 가장 직접적으로 영향을 받는 주요 이해관계자(예: 직원, 고객, 지역사회, 환경)는 누구입니까?

    2.  **GRI 표준 검토**: 아래에 제공된 GRI 표준 정의 목록과 그 본래의 목적을 검토합니다. 각 표준이 어떤 이해관계자를 대상으로 하는지 특히 주의 깊게 살펴보세요.

    3.  **정확한 매칭 및 근거 제시**: 1단계에서 분석한 '핵심 영향'과 '주요 이해관계자'를 바탕으로, 가장 직접적으로 관련된 GRI 항목을 선택합니다. 선택한 이유를 명확하게 작성하고, 간접적이거나 비유적인 연결은 피해주세요.

    ### 분석할 기업 활동 내용:
    {business_context}

    ### GRI 표준 정의 목록:
    {gri_items}

    ### 출력 형식:
    반드시 아래 JSON 형식에 맞춰서 답변해 주세요.
    {format_instructions}
    """

    parser = JsonOutputParser(pydantic_object=TopItemList)
    prompt = ChatPromptTemplate.from_template(
        template=top_level_prompt_template,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    llm = ChatOpenAI(model=model_name, temperature=0)
    chain = prompt | llm | parser

    # GRI 정의 목록을 프롬프트에 맞게 문자열로 변환
    gri_list_str = "\n".join([f"- GRI {item['code']} {item['name']}: {item['description']}" for item in gri_definitions])

    result = chain.invoke({
        "top_n": TOPN_SELECT,
        "gri_items": gri_list_str,
        "business_context": biz_context
    })
    print("--- 1단계 판별 완료 ---")
    return result

def run_stage2_find_sub_item(top_item: dict, gri_detail_data: dict, biz_context: str, embeddings: OpenAIEmbeddings) -> dict | None:
    """2단계: 특정 상위 항목에 대한 세부 항목을 RAG로 검색하고 평평한 구조의 dict로 반환합니다."""
    top_code = top_item.get('item_code')
    top_name = top_item.get('item_name')

    if not top_code or not top_name:
        print(f"▶ 경고: 상위 항목 정보가 불완전하여 건너뜁니다. 데이터: {top_item}")
        return None

    print(f"▶ 상위 항목 '{top_code} {top_name}'에 대한 세부 분석 시작...")
    sub_items_docs = find_sub_items(top_code, gri_detail_data)
    if not sub_items_docs:
        print(f" -> '{top_code} {top_name}'에 대한 세부 항목을 찾을 수 없습니다.")
        result_dict = {**top_item}
        result_dict['relevant_sub_items'] = []
        return result_dict

    vector_store = FAISS.from_documents(sub_items_docs, embeddings)
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    relevant_docs = retriever.invoke(biz_context)
    print(f" -> '{top_code} {top_name}' 분석 완료.")

    result_dict = {**top_item}
    result_dict['relevant_sub_items'] = [doc.page_content for doc in relevant_docs]
    return result_dict

# =================================================================
# 4) 메인 분석 파이프라인 함수
# =================================================================
def match_gri(report_raw: str, model_name: str, top_n: int) -> list:
    """
    기업의 원본 데이터(dict)를 입력받아 GRI 분석을 수행하고,
    최종 결과를 리스트 형태로 반환하는 메인 함수.
    """
    # ★★★★★ 여기가 핵심 수정 사항 (3) ★★★★★
    # --- 0. 분석에 필요한 데이터 로드 ---
    # gri_top_data = load_json(GRI_TOP_PATH) # -> 이제 전역 변수 GRI_TOP_DEFINITIONS 사용
    gri_detail_data = load_json(GRI_DETAIL_PATH)
    if gri_detail_data is None:
        raise FileNotFoundError("GRI 세부 데이터 파일을 찾을 수 없습니다.")

    biz_context = normalize(json.dumps(report_raw, ensure_ascii=False))

    # --- 1. 상위 GRI 항목 판별 ---
    top_level_result = run_stage1_find_top_items(biz_context, GRI_TOP_DEFINITIONS, MODEL_NAME, TOPN_SELECT)

    # print("\n" + "="*80 + "\n")

    # --- 2. 세부 항목 병렬 검색 ---
    print('='*100)
    print("Step5-4: 하위 GRI 항목 판별")
    print("--- 2단계: RAG를 통해 가장 관련성 높은 세부 항목 병렬 검색 시작 ---")
    embeddings = OpenAIEmbeddings()
    analysis_results = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_item = {
            executor.submit(run_stage2_find_sub_item, item, gri_detail_data, biz_context, embeddings): item
            for item in top_level_result.get('top_items', [])
        }
        for future in as_completed(future_to_item):
            try:
                result = future.result()
                if result:
                    analysis_results.append(result)
            except Exception as exc:
                item_info = future_to_item[future]
                print(f"'{item_info.get('item_name')}' 항목 분석 중 오류 발생: {exc}")

    return sorted(analysis_results, key=lambda x: x.get('item_code', ''))

# =================================================================
# 섹션 C: 성과 시각화 관련 함수들 (신규 추가)
# =================================================================


# 기존 GraphData 모델에 time_type 필드 추가
class GraphData(BaseModel):
    title: str = Field(description="그래프의 제목 (예: '수혜 인원 변화')")
    x_axis_label: str = Field(description="X축의 이름 (예: '연도', '단계', '목표')")
    y_axis_label: str = Field(description="Y축의 이름 (예: '수혜 인원 (명)')")
    x_values: List[str] = Field(description="X축에 해당하는 값들의 리스트")
    y_values: List[float] = Field(description="Y축에 해당하는 숫자 값들의 리스트")
    explanation: str = Field(description="추출된 시계열 데이터가 의미하는 바에 대한 간결한 한 줄 요약 설명")
    time_type: str = Field(description="시간축 유형: 'absolute' (절대시점), 'relative' (상대변화), 'progressive' (점진적 목표)")

class GraphInfo(BaseModel):
    graphs: List[GraphData] = Field(description="텍스트에서 추출된 그래프화 가능한 데이터 리스트. 데이터가 없으면 빈 리스트 []를 반환합니다.")

# 단일 시점 (KPI 요약용) 데이터 구조
class KeyMetric(BaseModel):
    metric_name: str = Field(description="핵심 지표의 이름 (예: '온실가스 감축량')")
    value: float = Field(description="지표의 숫자 값 (예: 6500)")
    unit: str = Field(description="지표의 단위 (예: 'tCO2e', '명', 'GWh')")
    context: str = Field(description="원본 텍스트에 언급된 전체 내용 (예: '연간 6,500tCO2e 감축')")
    explanation: str = Field(description="추출된 KPI가 의미하는 바에 대한 간결한 한 줄 요약 설명")

class KeyMetricList(BaseModel):
    metrics: List[KeyMetric] = Field(description="텍스트에서 추출된 핵심 성과 지표(KPI) 리스트")


# =================================================================
# ## 기능 함수 정의
# =================================================================
# --- ⚠️ 수정된 부분 1: 폰트 찾는 함수 개선 ---
def find_korean_font():
    """시스템에서 사용 가능한 한글 폰트를 찾아 경로를 반환합니다. (개선된 버전)"""
    # 1. Windows: '맑은 고딕'
    if os.name == 'nt':
        font_path = 'c:/Windows/Fonts/malgun.ttf'
        if os.path.exists(font_path):
            # print(f" 윈도우 '맑은 고딕' 폰트를 찾았습니다: {font_path}")
            return font_path

    # 2. macOS: 'AppleGothic'
    elif os.name == 'darwin':
        font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
        if os.path.exists(font_path):
            # print(f" macOS 'AppleGothic' 폰트를 찾았습니다: {font_path}")
            return font_path

    # 3. Linux/기타 환경: '나눔고딕' 또는 '맑은 고딕' 계열 폰트 검색
    print("-> 시스템에서 'Nanum' 또는 'Malgun' 계열 폰트를 검색합니다...")
    for font in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        try:
            # fm.FontProperties를 사용하여 폰트의 실제 이름을 가져옵니다.
            font_name = fm.FontProperties(fname=font).get_name()
            if 'nanum' in font_name.lower() or 'malgun' in font_name.lower():
                # print(f" 사용 가능한 한글 폰트를 찾았습니다: {font}")
                return font
        except RuntimeError:
            # 일부 잘못된 폰트 파일은 오류를 발생시킬 수 있어 예외 처리합니다.
            continue

    print("❌ 경고: 시스템에서 '맑은 고딕'이나 '나눔고딕'을 찾지 못했습니다. 그래프의 한글이 깨질 수 있습니다.")
    return None
# -----------------------------------------------------------------
# ### 시계열 변화 (그래프) 관련 함수들
# -----------------------------------------------------------------


def extract_graph_data(final_json_result: dict) -> Optional[List[dict]]:
    """[개선된 시계열] 절대시점과 상대변화를 구분하여 추출하는 함수"""
    print("--- 개선된 시계열 데이터 추출 시작 ---")
    text_to_analyze = "\n\n".join([f"## {k}\n{v}" for k, v in final_json_result.items() if isinstance(v, str)])

    parser = JsonOutputParser(pydantic_object=GraphInfo)

    prompt = ChatPromptTemplate.from_template(
        template="""당신은 정밀한 데이터 분석가입니다. 주어진 텍스트에서 시간의 흐름에 따른 정량적 변화 데이터를 찾되, **시간축의 성격을 정확히 구분**하여 추출해야 합니다.

## 🔍 시간축 분류 규칙 (매우 중요)

**1. ABSOLUTE (절대시점) - time_type: "absolute"**
- 조건: 각 수치에 명확한 연도/분기/월이 표기됨
- 예시: "2023년 460명 → 2024년 780명 → 2025년 1,200명"
- X축: 실제 연도/분기/월 사용
- 처리: 그대로 추출

**2. RELATIVE (상대변화) - time_type: "relative"**
- 조건: 시작→목표 형태이지만 구체적 시점이 없음
- 예시: "62% → 75%", "48% → 62% → 70%"
- X축: "현재", "1단계 목표", "2단계 목표", "3단계 목표" 등으로 표현
- ⚠️ **절대 임의의 연도를 만들어내지 마세요!**

**3. PROGRESSIVE (점진적 목표) - time_type: "progressive"**
- 조건: 3개 이상의 단계적 목표 수치 (특히 급여, 매출 등)
- 예시: "초임 평균 월 2.3M → 2.6M → 2.8M"
- X축: 점진적 향상을 나타내는 라벨
- 라벨 예시: ["1차년도", "2차년도", "3차년도"] or [현재, 1차 목표, 2차 목표,..., 최종목표]

## 📋 추출 지침
1. **최소 데이터 개수 규칙 (가장 중요)**: `y_values` 리스트에는 반드시 **2개 이상의 숫자 값**이 포함되어야 합니다. 데이터 포인트가 하나뿐인 경우는 **절대** 추출 대상이 아닙니다.

2. **데이터 분류 과정**:
   - 각 데이터를 위 3가지 중 하나로 분류
   - time_type 필드에 정확한 분류 기록

3. **X축 라벨링 규칙**:
   - ABSOLUTE: 원문의 연도/분기/월 그대로 사용
   - RELATIVE: "현재" → "목표" 또는 "1단계" → "2단계" → "최종단계" 또는 "1차년도" →  "2차년도" →  "3차년도"
   - PROGRESSIVE: 맥락에 맞는 점진적 표현 사용

4. **추출 대상**:
   - 2개 이상의 수치가 연결된 변화 데이터
   - **시간의 '변화'만 추출**: 반드시 동일한 지표가 시간(연도, 분기, 단계 등)에 따라 어떻게 변하는지를 보여주는 데이터만 추출해야 합니다.
   - 각 수치는 동일한 단위여야 함

5. **추출 금지**:
   - 단일 수치 ("연간 12,000 kWh")
   - 시간 흐름이 없는 비교 데이터

6. **`explanation` 작성 규칙 (매우 중요)**:
   - 원본 텍스트의 핵심 내용을 바탕으로, **시작 값과 최종 값을 반드시 포함**하는 한 문장의 설명으로 작성하세요.
   - 예시: "초임 평균이 월 2,300,000원에서 최종 목표 2,800,000원으로 상승하는 기대/목표/예정."


## 💡 예시

**입력**: "6개월 취업률 목표 62% → 75%"
**올바른 출력**:
```json
{{
  "title": "6개월 취업률 목표 변화",
  "x_axis_label": "목표 단계",
  "x_values": ["현재 목표", "개선 목표"],
  "y_values": [62, 75],
  "time_type": "relative"
}}
```

**입력**: "참여자 수: 2023년 460명 → 2024년 780명 → 2025년 1,200명"
**올바른 출력**:
```json
{{
  "title": "연도별 참여자 수 변화",
  "x_axis_label": "연도",
  "x_values": ["2023", "2024", "2025"],
  "y_values": [460, 780, 1200],
  "time_type": "absolute"
}}
```

[분석할 텍스트]
{text}

[출력 형식]
{format_instructions}""",
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    chain = prompt | llm | parser
    try:
        result = chain.invoke({"text": text_to_analyze})
        graphs_list = result.get('graphs', [])

        if graphs_list:
            print(f"--- 총 {len(graphs_list)}개의 그래프 데이터를 찾았습니다 ---")
            # 분류별 개수 출력
            absolute_count = sum(1 for g in graphs_list if g.get('time_type') == 'absolute')
            relative_count = sum(1 for g in graphs_list if g.get('time_type') == 'relative')
            progressive_count = sum(1 for g in graphs_list if g.get('time_type') == 'progressive')

            print(f"  - 절대시점(absolute): {absolute_count}개")
            print(f"  - 상대변화(relative): {relative_count}개")
            print(f"  - 점진적 목표(progressive): {progressive_count}개")

            return graphs_list
        else:
            print("--- 그래프로 표현할 시계열 데이터를 찾지 못했습니다 ---")
            return None

    except Exception as e:
        print(f"--- 오류: 데이터 추출 중 오류 발생 - {e} ---")
        return None

def extract_key_metrics(final_json_result: dict) -> Optional[List[dict]]:
    """[단일 시점] LLM을 사용해 KPI 데이터와 '한 줄 설명'을 명확한 규칙에 따라 추출합니다."""
    print("--- 8-1단계: 핵심 성과 지표(KPI) 추출 시작 ---")
    text_to_analyze = "\n\n".join([ f"## {k}\n{v}" for k, v in final_json_result.items() if isinstance(v, str) ])
    parser = JsonOutputParser(pydantic_object=KeyMetricList)
    prompt = ChatPromptTemplate.from_template(
        template="""당신은 ESG 컨설턴트입니다. 주어진 텍스트를 분석하여, 회사의 핵심적인 성과를 나타내는 **'시간 변화가 없는' 단일 정량 지표(KPI)**를 모두 찾아 JSON 리스트로 추출해야 합니다.

지침:
1. '연간', '총', '약' 등이 붙은 단일 숫자 성과를 모두 찾으세요.
2. **시간에 따른 변화가 있는 데이터는 반드시 제외**하세요.
3. 각 지표에 대해 'metric_name', 'value', 'unit', 'context'를 정확히 추출하세요.
4. 값(value)은 숫자만 남기고, 단위(unit)는 텍스트로 분리하세요.
5. **explanation 작성 규칙 (가장 중요)**
   원문 뉘앙스를 보고 반드시 다음 세 가지 형식 중 하나로 분류하여 설명을 작성합니다:
   - **목표치(Target)**: 원문에 "목표", "~할 것", "기대", "전망", "계획" 등의 표현이 있을 경우 -> **"목표치 — … 달성을 목표로 합니다."** 형식으로 작성
   - **성과치(Result)**: 원문에 "달성했다", "감소했다", "절감했다" 등 명확한 과거형 표현이 있을 경우 -> **"성과치 — …을 달성했습니다."** 형식으로 작성
   - **현황치(Status)**: 현재 상황이나 스펙을 설명하거나 위 두 조건에 해당하지 않을 경우 -> **"현황치 — …로 제시된 수치입니다."** 형식으로 작성
6. 추출할 데이터가 전혀 없으면, 'metrics' 필드를 빈 리스트 `[]`로 설정하여 반환해야 합니다.

[분석할 텍스트]
{text}
[출력 형식]
{format_instructions}""",
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    chain = prompt | llm | parser
    try:
        result = chain.invoke({"text": text_to_analyze})
        metrics_list = result.get('metrics', [])
        if metrics_list:
            print(f"--- 8-1단계: 총 {len(metrics_list)}개의 핵심 지표를 찾았습니다. ---")
            return metrics_list
        else:
            print("--- 8-1단계: 추출할 핵심 지표를 찾지 못했습니다. ---")
            return None
    except Exception as e:
        print(f"--- 8-1단계 오류: KPI 추출 중 오류 발생 - {e} ---")
        return None



def create_and_save_graphs_v3(graph_data: Optional[List[dict]], save_dir: str = 'result/graphs'):
    """[최종 시각화] v2에서 Y값 텍스트를 강조하여 가독성을 극대화한 버전"""
    if not graph_data:
        print("--- 시각화할 데이터가 없어 그래프 생성을 건너뜁니다 ---")
        return
    print('='*100)
    print("Step 7-2 최종 디자인으로 맞춤 그래프 생성 시작  ---")

    font_path = find_korean_font()
    font_prop = None
    if font_path:
        font_prop = fm.FontProperties(fname=font_path)
        # print(f" 한글 폰트 설정 완료: {font_prop.get_name()}")
    else:
        print("⚠️ 한글 폰트를 찾지 못했습니다.")

    os.makedirs(save_dir, exist_ok=True)

    for i, data in enumerate(graph_data):
        try:
            fig, ax = plt.subplots(figsize=(12, 8))
            fig.patch.set_facecolor('#F8F9FA')
            ax.set_facecolor('#F8F9FA')

            time_type = data.get('time_type', 'relative')

            colors = ['#ff8a80', '#80bfff', '#8aff80', '#ffd700', '#b19cd9']
            bar_colors = [colors[j % len(colors)] for j in range(len(data['x_values']))]

            bars = ax.bar(data['x_values'], data['y_values'], color=bar_colors, width=0.6, alpha=0.9)

            max_y_value = max(data['y_values']) if data['y_values'] else 1

            for bar, value, x_val, bar_color in zip(bars, data['y_values'], data['x_values'], bar_colors):

                # Y값(숫자)은 기존처럼 막대 바깥 상단에 표시 (유지)
                formatted_value = f'{value:,.0f}'
                unit_match = re.search(r'\((.*?)\)', data['y_axis_label'])
                unit = unit_match.group(1).strip() if unit_match else ''
                label_text = f'{formatted_value}{unit}'

                ax.text(bar.get_x() + bar.get_width()/2,
                        bar.get_height() + max_y_value * 0.02,
                        label_text, ha='center', va='bottom', fontsize=18, fontweight='bold',
                        color=bar_color, fontproperties=font_prop)

            # 제목, 설명 등은 기존과 동일
            type_label = {'absolute': '[연도별]', 'relative': '[목표 진행]', 'stage': '[단계별 목표]'}
            full_title = f"{type_label.get(time_type, '')} {data['title']}"
            ax.text(0.5, 1.08, full_title, transform=ax.transAxes, ha='center', va='top', fontsize=24, fontweight='bold', fontproperties=font_prop, color='#1A237E')
            props = dict(boxstyle='round,pad=0.5', facecolor='#E0F7FA', alpha=0.8, ec='none')
            ax.text(0.5, 1.0, data['explanation'], transform=ax.transAxes, ha='center', va='top', fontsize=13, fontproperties=font_prop, bbox=props, color='#006064')

            ax.set_ylim(0, max_y_value * 1.3)
            ax.set_xticks(range(len(data['x_values'])))

            # ##################################################################
            # ### ▼▼▼ X축 라벨 크기 및 굵기 수정 ▼▼▼                       ###
            # ##################################################################
            ax.set_xticklabels(
                data['x_values'],
                fontproperties=font_prop,
                fontsize=20,        # 15 -> 20 으로 크기 증가
                fontweight='heavy',   # 'bold' -> 'heavy' 로 더 두껍게
                color='#333'
            )
            # ##################################################################

            ax.set_yticks([])
            ax.spines[['top', 'right', 'left']].set_visible(False)
            ax.spines['bottom'].set_visible(True)
            ax.spines['bottom'].set_color('#CDCDCD')
            ax.tick_params(axis='x', length=0, pad=10)
            ax.grid(False)

            plt.tight_layout(rect=[0, 0, 1, 0.9])

            safe_filename = re.sub(r'[\\/*?:"<>|]', "", data['title'])
            save_path = os.path.join(save_dir, f"{time_type}_{i+1}_{safe_filename}.png")
            plt.savefig(save_path, dpi=200, bbox_inches='tight')
            plt.close()

            print(f"[{time_type.upper()}] 그래프 저장: '{save_path}'")

        except Exception as e:
            print(f"❌ '{data['title']}' 차트 생성 중 오류 발생: {e}")
            plt.close()

# print(" 모든 함수 및 클래스 정의 완료.")

def process_graph_data(final_json_data: dict):
    """[병렬 작업 1] Step 7: 시계열 데이터를 처리하고 그래프를 생성합니다."""
    print('='*100)
    print(" Step 7-1 (그래프 처리) 시작...")

    # 1. 먼저 데이터를 추출합니다.
    graphable_data = extract_graph_data(final_json_data)

    # 2. 데이터가 있는지 없는지부터 확인합니다.
    if not graphable_data:
        print(" Step 7: 시각화할 데이터가 없어 작업을 중단합니다.")
        return None # 데이터가 없으면 여기서 즉시 함수를 종료합니다.

    # 3. 데이터가 있는 것이 확인된 후에만 아래 작업을 수행합니다.
    # print(" Step 7: 그래프 데이터 추출 완료. 파일 저장 및 시각화를 시작합니다.")

    # 파일 저장
    with open('result/graph_data.json', 'w', encoding='utf-8') as f:
        json.dump(graphable_data, f, ensure_ascii=False, indent=4)
    print(" 'graph_data.json' 저장 완료.")

    # 누락되었던 그래프 생성 함수를 호출합니다.
    create_and_save_graphs_v3(graphable_data, save_dir='result/graphs')

    print("(그래프 처리) 성공적으로 완료.")
    return "Graph processing complete"




if __name__=='__main__':
    # =================================================================
    # ### 1단계: 문서 분석 및 JSON 보고서 생성
    # =================================================================
    company_info = read_document(DOCUMENT_PATH)
    masked_company_info = mask_info(company_info)
    company_summary = summary_sv(masked_company_info)
    sdgs_main = match_sdgs_by_llm(company_summary)
    with ThreadPoolExecutor(max_workers=8) as executor:
        future_sdgs_sub = executor.submit(match_sdgs_sub_by_llm, sdgs_main)
        future_raw_report = executor.submit(generate_raw_report, masked_company_info, sdgs_main)

        sdgs_sub = future_sdgs_sub.result()
        raw_report = future_raw_report.result()

    gri_main = match_gri(raw_report, MODEL_NAME, TOPN_SELECT)
    # ## 검증
    with ThreadPoolExecutor(max_workers=8) as executor:
        future_agent = executor.submit(process_raw_report_agent, raw_report)
        future_summary = executor.submit(summarize_validated_report, raw_report, sdgs_main)

        ai_explain, validated_report_list = future_agent.result()
        ai_summary = future_summary.result()

    final_json = to_json(company_summary, sdgs_main, sdgs_sub, gri_main,ai_explain, validated_report_list, ai_summary)
    with open('result/result.json', 'w', encoding='utf-8') as f:
        json.dump(final_json, f, ensure_ascii=False, indent=4)

    print("--- 1단계 완료: 최종 JSON 보고서가 생성되었습니다 ---\n")

    # =================================================================
    # ### 2단계: 최종 보고서 기반 데이터 추출 및 시각화
    # =================================================================
    print("--- 2단계: 데이터 추출 및 시각화 병렬 처리 시작 ---")

    # 1단계에서 생성된 final_json을 사용하여 병렬 처리 실행
    with ThreadPoolExecutor(max_workers=2) as executor:
        # Step 7 (그래프 처리)와 Step 8 (KPI 처리)을 동시에 실행
        future_graphs = executor.submit(process_graph_data, final_json)
        # future_kpis = executor.submit(process_kpi_data, final_json)

        # 두 작업이 모두 끝날 때까지 대기
        graph_result = future_graphs.result()
        # kpi_result = future_kpis.result()

    print("\n--- 모든 분석 및 시각화 작업이 완료되었습니다 ---")










